<a href="https://colab.research.google.com/github/carloscarvajal1cc/UTS_ML2019_ID13144148/blob/master/Copy_of_Decisiontree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Environment and constant preparation
import numpy as np
from sklearn.datasets import load_iris
import plotly.graph_objects as go
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

COLAB = True
IRIS_EXPERIMENT_X_SPACE_QUANTISATION_BIN_NUM = 50 # self-interpretation

def generate_all_X_space_normalised_samples_np(N):
    """
    :param N: X-space will be an N by N discrete-valued array
    """
    X0, X1 = np.meshgrid(np.arange(N), np.arange(N))
    return np.stack([X0.flatten(), X1.flatten()]).T / N

In [0]:
I_AM_RUNNING_THIS_NOTEBOOK_ON_MY_OWN_COMPUTER = False
COLAB = not I_AM_RUNNING_THIS_NOTEBOOK_ON_MY_OWN_COMPUTER

In [0]:
!pip install plotly --upgrade

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)


In [0]:
iris_db = load_iris()
all_x = generate_all_X_space_normalised_samples_np(
    IRIS_EXPERIMENT_X_SPACE_QUANTISATION_BIN_NUM)

In [0]:
# load_iris returns an object `iris_db`, but for now, we don't have much 
# information on the object. We usually start studying unknown objects
# in one of the following two steps.

# 1. check what is its `type`, and see what the author of the object template 
# (the class) has to say
print(type(iris_db))


<class 'sklearn.utils.Bunch'>


In [0]:
# Now let's get some help of the class. (most popular libraries
# are well documented)
iris_db?

In [0]:
dir(iris_db) # dir() lists methods / attributes of an object.

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [0]:
type(iris_db.data), type(iris_db.target)

(numpy.ndarray, numpy.ndarray)

In [0]:
print(iris_db.data.shape, iris_db.target.shape)

(150, 4) (150,)


In [0]:
print(iris_db.data[:10], iris_db.target[:10])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]] [0 0 0 0 0 0 0 0 0 0]


In [0]:
X, y = iris_db.data, iris_db.target

In [0]:

# So the idea becomes: 
# for each training sample, (let us identify the training sample 
# using an index i), 
# - include the data[i] and the target[i] in the subset, 
#   if target[i] is 0 or 1.

# translating the idea into a program
X_sub = []
y_sub = []
for i in range(len(iris_db.data)): 
    # [EXERCISE] What does range(len(...)) do 
    # for an ensemble object? 
    if iris_db.target[i] == 0 or iris_db.target[i] == 1:
        X_sub.append(iris_db.data[i])
        y_sub.append(iris_db.target[i])

In [0]:
print (len(X_sub))

100


In [0]:

# Check what we have obtained.
print(X_sub[55:57])
print(y_sub[55:57])

[array([5.7, 2.8, 4.5, 1.3]), array([6.3, 3.3, 4.7, 1.6])]
[1, 1]


In [0]:
X_sub = np.array(X_sub)
y_sub = np.array(y_sub)
print(X_sub[55:57])
print(y_sub[55:57])

[[0.51851852 0.33333333 4.5        1.3       ]
 [0.74074074 0.54166667 4.7        1.6       ]]
[1 1]


In [0]:

X_sub = []
y_sub = []
for x_, y_ in zip(iris_db.data, iris_db.target): 
    # [EXERCISE] Print the iteration variables in a zipped list.
    # I.e. construct two lists L1 and L2, and make a for loop
    # over "a, b in zip(L1, L2)", check the values of a and b.
    if y_ == 0 or y_ == 1:
        X_sub.append(x_)
        y_sub.append(y_)
X_sub = np.array(X_sub)
y_sub = np.array(y_sub)

In [0]:
print (y_sub[50:99])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]


In [0]:
# (*) We can use more "descriptive", less "instructive" construction
X_sub = np.array([x_ for x_, y_ in zip(iris_db.data, iris_db.target)
                  if y_ == 0 or y_ == 1])
y_sub = np.array([y_ for x_, y_ in zip(iris_db.data, iris_db.target)
                  if y_ == 0 or y_ == 1])

In [0]:
X_sub = X[(y==0) + (y==1)] # "+" for boolean OR
y_sub = y[(y==0) + (y==1)]

# Consolidate following reference to the data
X = X_sub
y = y_sub

In [0]:
print(((y==0) + (y==1)).shape)

(83,)


In [0]:
X = X[:, :2]

In [0]:
quant_num = IRIS_EXPERIMENT_X_SPACE_QUANTISATION_BIN_NUM

# A grid of cells to store the y-values in each area in
# X-space. 
data_bins = [[[] for a1 in range(quant_num)]
             for a2 in range(quant_num)]

# Decide which bin each training sample belongs to

# 1. We want the smallest value of an attribute to be stored in bin[0]
#    and largest in bin[49] (say, for 50 bins)
attrib0_min_value = X[:,0].min()
attrib0_max_value = X[:,0].max() # perform 0-1 normalisation on attribute 0
X[:, 0] = (X[:, 0] - attrib0_min_value) / (attrib0_max_value - attrib0_min_value)

attrib1_min_value = X[:,1].min()
attrib1_max_value = X[:,1].max()
X[:, 1] = (X[:, 1] - attrib1_min_value) / (attrib1_max_value - attrib1_min_value)

for x_, y_ in zip(X, y):
    attrib0_bin_index = int(round(x_[0] * (quant_num - 1)))
    attrib1_bin_index = int(round(x_[1] * (quant_num - 1)))
    data_bins[attrib0_bin_index][attrib1_bin_index].append(y_)
    # print(attrib0_bin_index, attrib1_bin_index)
    
# now we can arrange the data in a grid
X_quant = []
y_quant = []
for grid_row in range(quant_num):
    for grid_col in range(quant_num):
        this_bin = data_bins[grid_row][grid_col]
        if len(this_bin) > 0:
                X_quant.append((grid_row, grid_col))
                vote = int(round(np.mean(this_bin))) # if more than one y-value
                # has been allocated to this cell (small area in X-space),
                # we let them vote and take the majority.
                y_quant.append(vote)
                
X_quant = np.array(X_quant) / quant_num # (0..1)
y_quant = np.array(y_quant)

In [0]:

X = X_quant
y = y_quant

In [0]:
def preproc_data_for_complexity_experiment_quick(iris_db):
    
    X, y = iris_db.data, iris_db.target
    X = X[(y==0) + (y==1)][:, :2]
    y = y[(y==0) + (y==1)]
    
    # normalise to 0-1
    X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
    return X, y

def quantise_data(X, y, quant_num):
    
    # merge different y-values in the same cell                 
    X = np.round(X  * (quant_num - 1)).astype(np.int)
    data_bins = [[[] for a1 in range(quant_num)]
             for a2 in range(quant_num)]
    for (a1_, a2_), y_ in zip(X, y):
        data_bins[a1_][a2_].append(y_)
    X_quant = []
    y_quant = []
    for grid_row in range(quant_num):
        for grid_col in range(quant_num):
            this_bin = data_bins[grid_row][grid_col]
            if len(this_bin) > 0:
                    X_quant.append((grid_row, grid_col))
                    vote = int(round(np.mean(this_bin))) # if more than one y-value
                    # has been allocated to this cell (small area in X-space),
                    # we let them vote and take the majority.
                    y_quant.append(vote)
    X = np.array(X_quant) / quant_num # (0..1)
    y = np.array(y_quant)
    return X, y

In [0]:
X, y = preproc_data_for_complexity_experiment_quick(iris_db)
# you can comment out the following statement if not wanting quantisation
X, y = quantise_data(X, y, IRIS_EXPERIMENT_X_SPACE_QUANTISATION_BIN_NUM)

In [0]:
layout = go.Layout(
    xaxis=go.layout.XAxis(
        range=[0, 1],
        showgrid=True,
        zeroline=True,
        showline=True,
        gridcolor='#bdbdbd',
        gridwidth=1,
        zerolinecolor='#969696',
        zerolinewidth=2,
        linecolor='#636363',
        linewidth=2,
        mirror=True,
    ),
    yaxis=go.layout.YAxis(
        range=[0, 1],
        showgrid=True,
        zeroline=True,
        showline=True,
        gridcolor='#bdbdbd',
        gridwidth=1,
        zerolinecolor='#969696',
        zerolinewidth=2,
        linecolor='#636363',
        linewidth=2,
        mirror=True,
   ),
   height=600,
   width=600,
)

fig = go.Figure(
    data=[
        go.Scatter(
            x=all_x[:, 0], 
            y=all_x[:, 1], 
            marker_color="rgba(0.7, 0.7, 0.7, 0.3)",
            marker_size=6,
            marker_line_width=2,
            mode="markers",
            name="'All' X Space Samples"),
        go.Scatter(
            x=X[:, 0], 
            y=X[:, 1], 
            marker_color=y,
            marker_size=12,
            marker_line_width=2,
            mode="markers",
            name="Dataset Samples"), 
    ],
    layout=layout,
    layout_title_text="Quantized Simplified Iris Data"
)
if COLAB:
    fig.show(renderer="colab")
else:
    fig.show()


In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [0]:
train_num = 40
noise = 0.15

In [0]:
complex_E_in = []
complex_E_out = [] # we cannot compute this, as we don't have the true concept 
# of Iris data. We can make one up in a later version for experiment. But for now, 
# let us use the evaluation on held-out test data instead. See below complex_E_test.
complex_E_test = []
simple_E_in = []
simple_E_test = []

# We perform multiple rounds of experiments to test the statistics of 
# E_in and E_test for different models / experiment settings.

# Randomly add noise to the data.
rng = np.random.RandomState(42)
noisy_y = np.array([
    y_ if rng.rand() > noise else (1-y_)
    for y_ in y
])
for random_seed in range(500):
    # Randomly split training/test data
    X_train, X_test, y_train, y_test = train_test_split(
        X, noisy_y, train_size=train_num, test_size=len(y)-train_num,
        stratify=noisy_y,
        random_state=random_seed)
    
    # Fit a complex model
    dt_complex = DecisionTreeClassifier(max_depth=100)
    dt_complex.fit(X_train, y_train)
    
    # Record training and test error
    pred_on_X_train = dt_complex.predict(X_train)
    E_in = np.sum(pred_on_X_train != y_train) / len(y_train) # error rate
    complex_E_in.append(E_in)
    pred_on_X_test = dt_complex.predict(X_test)
    E_test = np.sum(pred_on_X_test != y_test) / len(y_test) # error rate
    complex_E_test.append(E_test)
    
    # Fit a simple model
    dt_simple = DecisionTreeClassifier(max_depth=2)
    dt_simple.fit(X_train, y_train)
    
    # Record training and test error
    pred_on_X_train = dt_simple.predict(X_train)
    E_in = np.sum(pred_on_X_train != y_train) / len(y_train) # error rate
    simple_E_in.append(E_in)
    pred_on_X_test = dt_simple.predict(X_test)
    E_test = np.sum(pred_on_X_test != y_test) / len(y_test) # error rate
    simple_E_test.append(E_test)

complex_E_in = np.array(complex_E_in)
complex_E_test = np.array(complex_E_test)
simple_E_in = np.array(simple_E_in)
simple_E_test = np.array( simple_E_test)

In [0]:
print("Complex Tree average E_in mean {:.3f}, std {:.3f}"
      .format(complex_E_in.mean(), complex_E_in.std()))
print("Complex Tree average E_out mean {:.3f}, std {:.3f}"
      .format(complex_E_test.mean(), complex_E_test.std()))
abs_diff = np.abs(complex_E_in - complex_E_test)
print("Complex Tree average |E_out - E_in| mean {:.3f}, std {:.3f}"
      .format(abs_diff.mean(), abs_diff.std()))

print("Simple Tree average E_in mean {:.3f}, std {:.3f}"
      .format(simple_E_in.mean(), simple_E_in.std()))
print("Simple Tree average E_out mean {:.3f}, std {:.3f}"
      .format(simple_E_test.mean(), simple_E_test.std()))
abs_diff = np.abs(simple_E_in - simple_E_test)
print("Simple Tree average |E_out - E_in| mean {:.3f}, std {:.3f}"
      .format(abs_diff.mean(), abs_diff.std()))

Complex Tree average E_in mean 0.000, std 0.000
Complex Tree average E_out mean 0.389, std 0.071
Complex Tree average |E_out - E_in| mean 0.389, std 0.071
Simple Tree average E_in mean 0.178, std 0.045
Simple Tree average E_out mean 0.289, std 0.062
Simple Tree average |E_out - E_in| mean 0.119, std 0.072
